In [39]:
import requests 
import pandas as pd
import numpy as np
import math
from pprint import pprint
import re

In [55]:
MAX_RECORDS = 200
latmin = 43.6406
latmax = 43.736439
lngmax = -79.252357       
lngmin = -79.549301
records = 2500
records_per_page = min(records, MAX_RECORDS)
pages = math.ceil(records / MAX_RECORDS)

maindf = pd.DataFrame({"Address":''},index=[np.arange(0, records)])
maindf["Latitude"]=""
maindf["Longitude"]=""
maindf["Sale Price"]= ""    
maindf['Mls Number']=""

for page in np.arange(pages):
    url = 'https://api2.realtor.ca/Listing.svc/PropertySearch_Post'
    body = {
        "CultureId":1,
        "ApplicationId":1,
        "PropertySearchTypeId":1,
        "TransactionTypeID":2,
        "BuildingTypeId":0,
        "LongitudeMin":f'{lngmin}',
        "LongitudeMax":f'{lngmax}',
        "LatitudeMin":f'{latmin}',
        "LatitudeMax":f'{latmax}',
        "PriceMin":250000,
        "RecordsPerPage": records_per_page,
        "CurrentPage": page + 1
    }

    response = requests.post(url, data=body).json() 
    
    for index, mls_post in enumerate(response['Results']):
        if page > 0:
            index += (MAX_RECORDS * page)
            
        try:                        
            lng = float(mls_post['Property']['Address']['Longitude'])
            address = mls_post['Property']['Address']
    
            if (lng >= lngmin and lng <= lngmax and address['AddressText'] not in maindf['Address'].unique()):
                maindf.loc[index, "Latitude"] = address['Latitude']  
                maindf.loc[index, "Longitude"] = address['Longitude']
                maindf.loc[index,"Address"] = address['AddressText']        
                maindf.loc[index,"Mls Number"]= mls_post['MlsNumber']
                maindf.loc[index,"Date Updated"] = mls_post['Individual'][0]['Organization']['PhotoLastupdate']
                    
                try:
                    maindf.loc[index,"Sale Price"] = mls_post['Property']['Price']
                except:
                    maindf.loc[index,"Sale Price"] = ""     
                
        except Exception as e:
            print('Error ', e)  
            
maindfclean = maindf.loc[maindf['Address']!="",:]


In [56]:
maindfclean[maindfclean['Address'] == ""]
maindfclean['Address'].count()

2445

In [57]:
maindfclean = maindf.loc[maindf['Address']!="",:]
maindfclean.set_index('Mls Number')

,Address,Latitude,Longitude,Sale Price,Date Updated
Mls Number,,,,,
W4615950,"#503 -100 LOTHERTON PTWY W|Toronto, Ontario M6...",43.7064429,-79.467532,"$285,000",2019-11-05 8:24:35 AM
C4623255,"#904 -3555 BATHURST ST|Toronto, Ontario M6A2Y8",43.7307312,-79.4319228,"$299,000",2019-11-01 2:47:00 PM
C4619564,"#1110 -720 SPADINA AVE|Toronto, Ontario M5S2T9",43.6654303191678,-79.4036489725113,"$299,800",2019-10-31 2:45:58 PM
E4588023,"#419 -1 MASSEY SQ|Toronto, Ontario M4C5L4",43.6964446,-79.2941767,"$299,900",2019-10-22 8:55:40 AM
W4630920,"#1006 -100 LOTHERTON PKWY|Toronto, Ontario M6B2G8",43.7064429,-79.467532,"$309,000",2019-11-10 6:22:57 PM
E4628867,"#2502 -3 MASSEY SQ|Toronto, Ontario M4C5L5",43.6968278,-79.2934124,"$310,000",2019-11-07 6:47:02 PM
W4461841,"#1002 -50 LOTHERTON PTWY|Toronto, Ontario M6B2G7",43.7058446,-79.4669622,"$320,000",2019-10-22 9:56:13 AM
C4622029,"#1612 -10 SUNNY GLENWAY WAY|Toronto, Ontario M...",43.710003,-79.3311344,"$320,000",2019-10-31 2:46:22 PM
E4587829,"#2218 -1 MASSEY SQ|Toronto, Ontario M4C5L4",43.6964446,-79.2941767,"$324,900",2019-10-29 6:44:37 PM


In [58]:
maindfclean.count()

Address         2445
Latitude        2445
Longitude       2445
Sale Price      2445
Mls Number      2445
Date Updated    2445
dtype: int64

In [59]:
pattern = re.compile(r'(\d\d\d\d-\d\d-\d\d)')

In [60]:
maindfclean['Date Updated'] = maindfclean['Date Updated'].str.extract(pattern)


C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
maindfclean

,Address,Latitude,Longitude,Sale Price,Mls Number,Date Updated
0,"#503 -100 LOTHERTON PTWY W|Toronto, Ontario M6...",43.7064429,-79.467532,"$285,000",W4615950,2019-11-05
1,"#904 -3555 BATHURST ST|Toronto, Ontario M6A2Y8",43.7307312,-79.4319228,"$299,000",C4623255,2019-11-01
2,"#1110 -720 SPADINA AVE|Toronto, Ontario M5S2T9",43.6654303191678,-79.4036489725113,"$299,800",C4619564,2019-10-31
3,"#419 -1 MASSEY SQ|Toronto, Ontario M4C5L4",43.6964446,-79.2941767,"$299,900",E4588023,2019-10-22
4,"#1006 -100 LOTHERTON PKWY|Toronto, Ontario M6B2G8",43.7064429,-79.467532,"$309,000",W4630920,2019-11-10
5,"#2502 -3 MASSEY SQ|Toronto, Ontario M4C5L5",43.6968278,-79.2934124,"$310,000",E4628867,2019-11-07
6,"#1002 -50 LOTHERTON PTWY|Toronto, Ontario M6B2G7",43.7058446,-79.4669622,"$320,000",W4461841,2019-10-22
7,"#1612 -10 SUNNY GLENWAY WAY|Toronto, Ontario M...",43.710003,-79.3311344,"$320,000",C4622029,2019-10-31
8,"#2218 -1 MASSEY SQ|Toronto, Ontario M4C5L4",43.6964446,-79.2941767,"$324,900",E4587829,2019-10-29
9,"#506 -50 LOTHERTON PTWY|Toronto, Ontario M6B2G7",43.7058446,-79.4669622,"$325,000",W4550339,2019-10-22
